In [1]:
#default_exp cli
from nbdev import *
# %nbdev_default_export cli

# cli

> This is a cli module for RemoteConnect

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastcore.script import call_parse, Param, bool_arg
from fastcore.basics import in_colab
from RemoteConnect.core import RemoteCode, RemoteJupyter, RemotePluto, mount_drive, connect_to_ngrok

## Start Code

In [ ]:
#export
@call_parse
def start_code(port:Param("Port to Start Code", type=int)=10000, 
               password:Param("Password to Start Code", type=str)=None, 
               tunnel:Param("Tunel Type", type=str)='ngrok',
               authtoken:Param("Tunnel Authtoken for ngrok", type=str)=None):
    "Starts Code Server"
    if in_colab(): mount_drive()
    remote = RemoteCode(password=password, port=port, tunnel=tunnel, authtoken=authtoken)
    remote.launch()

## Start Jupyter

In [ ]:
#export
@call_parse
def start_jupyter(port:Param("Port to Start Jupyter", type=int)=9000, 
                  ui:Param("Interface to start", type=str)='notebook',
                  tunnel:Param("Tunel Type", type=str)='ngrok',
                  authtoken:Param("Tunnel Authtoken for ngrok", type=str)=None,
                  subdomain:Param("Add a subdomain only for localtunnel", type=str)=None,
                  install_code:Param("Flag to install code", type=bool)=False,
                  install_julia:Param("Flag to install code", type=bool)=False,
                  ):
    "Starts Jupyter"
    if in_colab(): 
        print("Mounting Drive")
        mount_drive()
    remote = RemoteJupyter(port=port, 
                           ui=ui, 
                           tunnel=tunnel, 
                           authtoken=authtoken,
                           subdomain=subdomain,
                           install_code=install_code, 
                           install_julia=install_julia
                          )
    remote.launch()

In [ ]:
# start_jupyter(tunnel="telebit")

## Start Pluto

In [ ]:
#export
@call_parse
def start_pluto(port:Param("Port to Start Jupyter", type=int)=9000,
                tunnel:Param("Tunel Type", type=str)='ngrok',
                authtoken:Param("Tunnel Authtoken for ngrok", type=str)=None):
    "Starts Pluto.jl reactive notebook"
    if in_colab(): mount_drive()
    remote = RemotePluto(port=port, tunnel=tunnel, authtoken=authtoken)
    remote.launch()